In [1]:
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras import applications
from tensorflow.keras.layers import BatchNormalization,Conv2D, AveragePooling2D,TimeDistributed,Dense, Dropout, Activation, Flatten,GlobalAveragePooling2D,MaxPool2D

In [9]:
train_path='/home/paa/COVID-19/dataset'
train_datagen=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input,validation_split = 0.25) #included in our dependencies

train_generator=train_datagen.flow_from_directory(train_path,
                                                 target_size=(500,500),
                                                 color_mode='grayscale',
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 subset = 'training',
                                                 shuffle=True)

validation_generator = train_datagen.flow_from_directory(train_path,
                                                 target_size=(500,500),
                                                 color_mode='grayscale',
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 subset = 'validation',
                                                 shuffle=True)

step_size_train=train_generator.n//train_generator.batch_size
step_size_validation=validation_generator.n//validation_generator.batch_size
print(step_size_train)
print(step_size_validation)

Found 1350 images belonging to 2 classes.
Found 450 images belonging to 2 classes.
42
14


In [5]:
img_height,img_width=500,500
base_model = applications.densenet.DenseNet121(weights= None, include_top=False, input_shape= (img_height,img_width,1))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation= 'sigmoid')(x)
model = Model(inputs = base_model.input, outputs = predictions)
model.compile(optimizer= 'RMSprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 500, 500, 1) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 506, 506, 1)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 250, 250, 64) 3136        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 250, 250, 64) 256         conv1/conv[0][0]                 


conv4_block6_1_bn (BatchNormali (None, 31, 31, 128)  512         conv4_block6_1_conv[0][0]        
__________________________________________________________________________________________________
conv4_block6_1_relu (Activation (None, 31, 31, 128)  0           conv4_block6_1_bn[0][0]          
__________________________________________________________________________________________________
conv4_block6_2_conv (Conv2D)    (None, 31, 31, 32)   36864       conv4_block6_1_relu[0][0]        
__________________________________________________________________________________________________
conv4_block6_concat (Concatenat (None, 31, 31, 448)  0           conv4_block5_concat[0][0]        
                                                                 conv4_block6_2_conv[0][0]        
__________________________________________________________________________________________________
conv4_block7_0_bn (BatchNormali (None, 31, 31, 448)  1792        conv4_block6_concat[0][0]        
__________

conv5_block14_1_bn (BatchNormal (None, 15, 15, 128)  512         conv5_block14_1_conv[0][0]       
__________________________________________________________________________________________________
conv5_block14_1_relu (Activatio (None, 15, 15, 128)  0           conv5_block14_1_bn[0][0]         
__________________________________________________________________________________________________
conv5_block14_2_conv (Conv2D)   (None, 15, 15, 32)   36864       conv5_block14_1_relu[0][0]       
__________________________________________________________________________________________________
conv5_block14_concat (Concatena (None, 15, 15, 960)  0           conv5_block13_concat[0][0]       
                                                                 conv5_block14_2_conv[0][0]       
__________________________________________________________________________________________________
conv5_block15_0_bn (BatchNormal (None, 15, 15, 960)  3840        conv5_block14_concat[0][0]       
__________

In [ ]:
graph=model.fit_generator(generator=train_generator,
                    validation_data=validation_generator,
                   steps_per_epoch=step_size_train,
                    validation_steps=step_size_validation,
                   epochs=1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(graph.history['acc'])
plt.plot(graph.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(graph.history['loss'])
plt.plot(graph.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [2]:
model=tf.keras.models.load_model("covid19 densenet02.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
model.summary()
def prepare(ima):
    IMG_SIZE = 500  # 50 in txt-based
    img_array = cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    img_array=img_array/255.0  # filepathread in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1,IMG_SIZE, IMG_SIZE,1)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 500, 1) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 506, 506, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 250, 250, 64) 3136        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 250, 250, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [6]:
import cv2
for i in range(1,40):
    img=cv2.imread("/home/paa/COVID-19/test/"+str(i)+"N.jpeg")
    a=model.predict(prepare(img))
    print(a)

[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(validation_generator, 452 // 5+1)
# y_pred = np.argmax(Y_pred, axis=1)
Y_pred[Y_pred>0.5]=1
Y_pred[Y_pred<0.5]=0
y_pred=Y_pred
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['Normal', 'Positive']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [12]:
img=cv2.imread("/home/paa/COVID-19/dataset/Normal/105.jpeg")
a=model.predict(prepare(img))
print(a)

[[0.]]
